# Анализ данных

Программа анализирует две группы данных, выбранных ранее

Рассчитываются статистически значимые различия в концентрации белков между двумя группами\
В результате программы строится Volcano-plot и сохраняются стат. значимо различающиеся белки


In [15]:
import numpy as np
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
import plotly.express as px
import xlsxwriter
import pandas as pd
import os
from Bio import SeqIO
import urllib.request
from bioservices import UniProt
import io
%matplotlib inline

In [16]:
### Загружаем данные из сводной таблицы с группами

path='all Tables/Combined_contamination.xlsx'

xl = pd.ExcelFile(path)
sh_n=xl.sheet_names

# Разбиенте на группы. Можно сколько угодно групп, при анализе вписываются нужные 2 группы

g1 = pd.read_excel(path,sheet_name=sh_n[0], index_col='Majority protein IDs', usecols='B,D:Q') 
g2 = pd.read_excel(path,sheet_name=sh_n[0], index_col='Majority protein IDs', usecols='B,R:AD')


# Колонки, отвечающие за контаминацию

tops = pd.read_excel(path,sheet_name=sh_n[0], index_col='Majority protein IDs', usecols='B,AE:AF')


df1 = pd.DataFrame(g1)
df2 = pd.DataFrame(g2)

tps = pd.DataFrame(tops)

In [17]:
# Заменяем все '0' на 'NaN', считая, что белок не определился в запуске 
# Для того, чтобы потом не считать '0' в среднем значении

df1.replace('n/d', np.nan, inplace=True)
df2.replace('n/d', np.nan, inplace=True)


In [18]:
#Проверка - смотрим на какие образцы находятся в группе

df2.iloc[0]

Sample data__# per plt 15    68844.326924
Sample data__# per plt 16    71672.573490
Sample data__# per plt 17    69250.028142
Sample data__# per plt 18    65953.871478
Sample data__# per plt 19    85664.405467
Sample data__# per plt 20    95633.089903
Sample data__# per plt 21    94599.740896
Sample data__# per plt 22    80611.534580
Sample data__# per plt 23    91199.388589
Sample data__# per plt 24    90979.528897
Sample data__# per plt 25    91931.822291
Sample data__# per plt 26    86473.285933
Sample data__# per plt 27    89572.080560
Name: CON__P00761, dtype: float64

In [19]:
# Определение стат. значимо различающихся белков, которые определились >3  запусках в каждой группе

pv, ratio, mean_g1, mean_g2, names,top1,top2 = [], [], [], [], [],[],[]

for i in range(0,len(df1)):
    a=df1.iloc[i]
    b=df2.iloc[i]
    t1 = tps.iloc[i,0]
    t2 = tps.iloc[i,1]
    count1=a.count()
    count2=b.count()
    if (count1>3)&(count2>3):
        t_stat, p = ttest_ind(a,b,nan_policy='omit')
        m1=a.mean()
        m2=b.mean()
        mean_g1=np.append(mean_g1,m1)
        mean_g2=np.append(mean_g2,m2)
        pv=np.append(pv,-np.log10(p))
        ratio=np.append(ratio,np.log2(m1/m2))
        names=np.append(names,a.name)
        top1=np.append(top1,t1)
        top2=np.append(top2,t2)

In [20]:
# Построение Volcano-plot

dfnew = pd.DataFrame(list(zip(ratio, pv,names,mean_g1,mean_g2)),columns =['log(Ratio)', '-log(p-value)','name','Mean1','Mean2'])

        
fig = px.scatter(dfnew, x='log(Ratio)',y='-log(p-value)',color=np.log10(dfnew['Mean1']),hover_name='name',hover_data=['Mean1', 'Mean2','name'])
# fig.add_shape(type="rect",
#     x0=-1.2, y0=1.3, x1=1.2, y1=3.5,
#     line=dict(color="RoyalBlue"),
# )
fig.update_layout(
    title="Volcano plot",
    xaxis_title='log(ratio)',
    yaxis_title='-log(p-value)')
fig.show()       

In [21]:
## Описание различающихся белков
# Может занять около 10 мин

service = UniProt()
rt1,rt2,rt3, pvv,mg1,mg2,rto,nms,tt1,tt2 = [],[],[],[],[],[],[],[],[],[]



for i in range(len(names)-1):
    if ((ratio[i]<-1.0) or (ratio[i]>1.0)) and (pv[i]>1.3):
        pvv.append(pow(10, -pv[i]))
        rto.append(mean_g1[i]/mean_g2[i])
        mg1.append(mean_g1[i])
        mg2.append(mean_g2[i])
        nms.append(names[i])
        tt1.append(top1[i])
        tt2.append(top2[i])
        try:
            a=service.search(names[i]+'+and+taxonomy_id:9606', frmt="tsv", limit=3, columns='id')
            b=service.search(names[i]+'+and+taxonomy_id:9606', frmt="tsv", limit=3, columns='protein_name')
            try:
                c = urllib.request.urlopen('http://www.uniprot.org/uniprot/'+names[i]+'.xml')
                record = SeqIO.read(c, 'uniprot-xml')
                s=record.annotations['comment_function'][0]
            except:
                 s='NaN'
            try:
                rt1.append(list(a.split("\n"))[1])
                rt2.append(list(b.split("\n"))[1])
                rt3.append(s)
            except:
                rt1.append('NaN1')
                rt2.append('NaN1')
                rt3.append('NaN1')
        except:
            rt1.append('NaN')
            rt2.append('NaN')
            rt3.append('NaN')


dfff = pd.DataFrame(list(zip(nms,rt1, rt2,rt3,mg1, mg2, rto, pvv,tt1,tt2)),
               columns =['Accession','Entry name', 'Protein names','Description', 'Mean 1', 'Mean 2','Ratio','p-value', 'Top 100 RBC','Top 100 plasma' ])


In [22]:
# Сохранение отличающихся белков в виде таблицы 

dfff.to_excel(r'Results/Signific different(g1 vs g2.xlsx')

In [23]:
# Сохранение средних значений и значимостей для всех белков групп для построения Volcano-plot

dfnew.to_excel(r'Results/all_mean_g1_g2.xlsx')